In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()

In [2]:
Modes = 4
FWT_DoFs = 1
DoFs = Modes + FWT_DoFs

In [3]:
p = ma.DynamicModelParameters(DoFs)
p.DoFs  = ma.ModelSymbol(value = DoFs,string = 'DoFs')                          # number of degrees of freedom

#gravity vector
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelVector(value =[0,0,-1],length=3,string='g_v') # The stationary point

# Reduced order matrices
p.M = ma.ModelVector(value =[0]*Modes,length=Modes,string='M', real=True)       # Generalised Mass Matrix of Inner Wing
p.K = ma.ModelVector(value =[0]*Modes,length=Modes,string='K', real=True)       # Generalised Stiffness Matrix of Inner Wing
p.S = ma.ModelMatrix(size=(3,Modes),string = 'S', real=True)               
p.R = ma.ModelMatrix(size=(3,Modes),string = 'R', real=True)   

#hinge properties
p.X_h = ma.ModelVector(value =[0,1,0],length=3,string='X_h')        # Location of hinge in wing frame
p.k_h = ma.ModelSymbol(value = 0,string = 'k_h')                    # Hinge Stiffness
p.d_h = ma.ModelSymbol(value = 0,string = 'd_h')                    # Hinge Damping             

# FWT properties
p.Lambda = ma.ModelSymbol(value = 0.3,string = 'Lambda')            # Flare Angle
p.m_f = ma.ModelSymbol(value = 0,string = 'm_f')                    # FWT mass
p.I_f = ma.ModelVector(value =[0,0,0],length=3,string='I_f')        # FWT moments of inertia
p.X_f = ma.ModelVector(value =[1,1,0],length=3,string='X_f')        # location of FWT CoM relative to hinge semi-chord

Xb = ma.VarVector('Xb',3,1)

In [4]:
# functions that represent rotation of the hinge due to inner wing bending
Ry = sym.Function('Ry')(t)
Rx = sym.Function('Rx')(t)
R = p.R*sym.Matrix(p.q[:Modes])
R = sym.eye(3) + ma.Wedge(R)
R

⎡                    1                       -R_2,0⋅q₀ - R_2,1⋅q₁ - R_2,2⋅q₂ - ↪
⎢                                                                              ↪
⎢R_2,0⋅q₀ + R_2,1⋅q₁ + R_2,2⋅q₂ + R_2,3⋅q₃                       1             ↪
⎢                                                                              ↪
⎣-R_1,0⋅q₀ - R_1,1⋅q₁ - R_1,2⋅q₂ - R_1,3⋅q₃  R_0,0⋅q₀ + R_0,1⋅q₁ + R_0,2⋅q₂ +  ↪

↪  R_2,3⋅q₃  R_1,0⋅q₀ + R_1,1⋅q₁ + R_1,2⋅q₂ + R_1,3⋅q₃ ⎤
↪                                                      ⎥
↪            -R_0,0⋅q₀ - R_0,1⋅q₁ - R_0,2⋅q₂ - R_0,3⋅q₃⎥
↪                                                      ⎥
↪ R_0,3⋅q₃                       1                     ⎦

In [5]:
# generate frames of reference
body_frame = ma.frames.HomogenousFrame().simplify()
v = p.X_h + p.S*sym.Matrix(p.q[:Modes])

# hinge_frame = body_frame.Translate(*v).R_x_small(Rx).R_y_small(R[1]).R_z(p.Lambda).R_x(-p.q[-1]).simplify()
hinge_frame = body_frame.Translate(*v).Rotate(R).R_z(p.Lambda).R_x(-p.q[-1]).simplify()
fwt_frame = hinge_frame.R_z(-p.Lambda)

In [6]:
#Create Elemnts
M = ma.elements.MassMatrix(p.m_f,p.I_f[0],p.I_f[1],p.I_f[2])


## Inner Wing Modal Element
wing_ele = ma.elements.ModalElement(p.q, body_frame, p.q[:Modes], p.M, p.K, name='wing',simplify = False)
## FWT mass element
fwt_elements = []
gv = sym.Matrix([*(p.g_v*p.g)])
fwt_elements.append(ma.elements.RigidElement(p.q, fwt_frame, M, grav_vec=gv, com_pos=p.X_f, name='fwt',simplify = False,alt_method=True))
fwt_elements.append(ma.elements.Spring(p.q,p.q[-1],p.k_h,name="Hinge Spring"))
fwt_elements.append(ma.elements.Damper(p.q,p.q[-1],p.d_h,name="Hinge Damper"))

In [8]:
# generate EoMs
sm = ma.SymbolicModel.FromElementsAndForces(p.q,[*fwt_elements,wing_ele])
# sm.M

Generating EoM for Element 1 out of 4 - fwt:RigidElement
Generating EoM for Element 2 out of 4 - Hinge Spring:Spring
Generating EoM for Element 3 out of 4 - Hinge Damper:Damper
Generating EoM for Element 4 out of 4 - wing:ModalElement


In [9]:
R = p.R*sym.Matrix(p.q[:Modes])
Rd = p.R*sym.Matrix(p.q[:Modes])
rot_rep = {Rx.diff(t):Rd[0],Ry.diff(t):Rd[1],
          Rx:R[0],Ry:R[1]}
sm = sm.msubs(rot_rep)
fwt_frame = fwt_frame.msubs(rot_rep)

In [10]:
funcs = []
Xi = ma.VarVector('Xi',3,1)
funcs.append(('get_fwt_V_b',fwt_frame.A.T*fwt_frame.transform_point(Xi).diff(t)))
funcs.append(('get_fwt_V_global',fwt_frame.transform_point(Xi).diff(t)))
funcs.append(('get_pos_global2fwt',fwt_frame.transform_global_point(Xi)))
funcs.append(('get_pos_fwt2global',fwt_frame.transform_point(Xi)))
# funcs.append(('get_fwt_V_b_body',body_frame.A.T*fwt_ele.frame.transform_point(X).diff(t)))
funcs.append(('get_fwt_A',fwt_frame.A))
funcs.append(('get_body_A',body_frame.A))
funcs.append(('get_hda',R[0]))
funcs.append(('get_fold',p.q[-1]))
funcs.append(('get_fold_dt',p.qd[-1]))

In [11]:
#create function for an arbitary force on FWT
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')
W_n = sym.Matrix([F_x,F_y,F_z,M_x,M_y,M_z])
W = ma.VarVector('W',6,1)
fwt_Q = fwt_frame.Translate(*Xi).BodyJacobian(p.q).T*W_n
fwt_Q = me.msubs(fwt_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
#create function for an arbitary force on FWT
funcs.append(('get_fwt_Q',fwt_Q))

In [12]:
base_dir = "C:/git/ROM_CL_FWT/src_matlab"
class_name = f'ROM_M{Modes}_FWT_RC'
base_class = "mbd.BaseRC"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable Xi found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_global. It will be added to the function signature.
Unknown variable Xi found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable Xi found in function get_pos_fwt2global. It will be added to the function signature.
Unknown variable W found in function get_fwt_Q. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_Q. It will be added to the function signature.
